In [1]:
import cv2
import imutils
import pandas as pd
import numpy as np

from skimage import exposure
from skimage.filters import threshold_local

In [2]:
def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 1000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.015 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
    return biggest

In [3]:
def question_cnts(contours):
    questionCnts = []
    if len(contours) > 0:
        for c in cnts: 
            # compute the bounding box of the contour, then use the bounding box to derive the aspect ratio
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)
            # in order to label the contour as a question, region should be sufficiently wide, sufficiently tall,
            # and have an aspect ratio approximately equal to 1
            if w >= 10 and h >= 10 and ar >= 0.8  and ar <= 1.2:
                questionCnts.append(c)

    return questionCnts

In [4]:
image = cv2.imread('Resources/bubble_sheet.png')
(h, w, d) = image.shape
ratio = 300.0 / w
dim = (300, int(h*ratio))
img = cv2.resize(image, dim)
img_original = img.copy()

gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
blurred = cv2.bilateralFilter(gray, 20, 30, 30)
edged = cv2.Canny(blurred, 75, 200)

## Find contours of paper

In [5]:
# RETR_EXTERNAL retrieves only the extreme outer contours
# cv2.CHAIN_APPROX_SIMPLE: This is the contour approximation method
cnts, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# select 10 biggest contours
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]

# Now lets search for the biggest contour that has 4 corners
biggest = biggest_contour(cnts)

cv2.drawContours(img, [biggest], -1, (0, 255, 0), 2)

array([[[ 50,  74,  85],
        [ 52,  78,  89],
        [ 54,  80,  91],
        ...,
        [ 66,  91, 101],
        [ 62,  88,  98],
        [ 61,  87,  98]],

       [[ 52,  78,  89],
        [ 49,  74,  85],
        [ 51,  77,  88],
        ...,
        [ 70,  94, 104],
        [ 59,  85,  96],
        [ 70,  94, 104]],

       [[ 48,  72,  83],
        [ 50,  75,  86],
        [ 52,  78,  89],
        ...,
        [ 59,  85,  96],
        [ 68,  93, 103],
        [ 60,  86,  97]],

       ...,

       [[ 49,  74,  85],
        [ 41,  63,  73],
        [ 41,  63,  74],
        ...,
        [ 58,  84,  95],
        [ 67,  91, 102],
        [ 58,  84,  95]],

       [[ 46,  70,  81],
        [ 50,  75,  86],
        [ 45,  69,  79],
        ...,
        [ 54,  80,  91],
        [ 57,  83,  94],
        [ 58,  84,  95]],

       [[ 47,  70,  81],
        [ 45,  68,  79],
        [ 49,  74,  85],
        ...,
        [ 60,  86,  97],
        [ 53,  79,  90],
        [ 65,  90, 101]]

In [6]:
# Image shape modification for hstack
blurred = np.stack((blurred,) * 3, axis=-1)
edged = np.stack((edged,) * 3, axis=-1)

print(image.shape)
print(blurred.shape)
print(edged.shape)

(700, 525, 3)
(400, 300, 3)
(400, 300, 3)


In [7]:
img_hor = np.hstack((img_original, blurred, edged, img))
cv2.imshow("Contour detection", img_hor)
cv2.waitKey(0)

-1

# Determine corners

In [8]:
# 1) Reshape contour points to have 4 lists with 2 places in the list. 
# It represents our 4 corners with x and y cordinates.
points = biggest.reshape(4, 2) 
# we create empty array where we will store coordinates in the correct order.
input_points = np.zeros((4, 2), dtype="float32")

In [9]:
# we want to mantain the same corner points order as we used before:
# top_left, top_right, bottom_left, bottom_right

# top_left and bottom right coordinates we can get with sum of the x and y coodinates
points_sum = points.sum(axis=1)
# top left point will have the smallest sum
input_points[0] = points[np.argmin(points_sum)]
# bottom right will have the biggest sum
input_points[3] = points[np.argmax(points_sum)]

# top right and bottom left points we can get by taking the difference between x and y cords
points_diff = np.diff(points, axis=1)
# top right point will have the smallest difference
input_points[1] = points[np.argmin(points_diff)]
# bottom left will have the biggest difference
input_points[2] = points[np.argmax(points_diff)]

In [10]:
# Now we have to calculate the dimensions of our new image
# We will calculate the distance between points coordinates
(top_left, top_right, bottom_left, bottom_right) = input_points
# calculate the bottom width of the image by computing the dist between x and y cords of the bot right and bot left points
bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
# the same logic for next 3 widths
top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
right_width = np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + ((top_right[1] - bottom_right[1]) ** 2))
left_width = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

In [11]:
# Define output image size
# As we don't know the exact dimention, we will use the maximum width and hight
max_width = max(int(bottom_width), int(top_width))
# max_height = max(int(right_height), int(left_height))
max_hight = int(max_width * 1.414) # for A4

## Perspective transformation

In [12]:
# Desired points values in the output image
converted_points = np.float32([[0, 0], [max_width, 0], [0, max_hight], [max_width, max_hight]])

In [13]:
# Perspective transformation
matrix = cv2.getPerspectiveTransform(input_points, converted_points)
img_output = cv2.warpPerspective(img_original, matrix, (max_width, max_hight))

In [14]:
gray = cv2.cvtColor(img_output, cv2.COLOR_BGR2GRAY)
# blurred = cv2.bilateralFilter(gray, 10, 10, 10)
# edged = cv2.Canny(blurred, 100, 200)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [15]:
# Image shape modification for hstack
stack_im = np.stack((thresh,) * 3, axis=-1)

print(img_output.shape)
print(stack_im.shape)

(263, 186, 3)
(263, 186, 3)


In [16]:
# define the answer key which maps the question number to the correct answer
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [17]:
warped_copy = img_output.copy()

In [18]:
cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
questionCnts = question_cnts(cnts)
cv2.drawContours(warped_copy, questionCnts, -1, (0, 255, 0), 2)

array([[[ 89, 111, 119],
        [ 93, 114, 122],
        [ 93, 114, 122],
        ...,
        [235, 239, 238],
        [238, 242, 240],
        [234, 238, 239]],

       [[193, 202, 203],
        [192, 201, 203],
        [195, 203, 205],
        ...,
        [234, 238, 236],
        [238, 242, 240],
        [228, 233, 234]],

       [[202, 211, 212],
        [196, 207, 209],
        [200, 210, 212],
        ...,
        [233, 237, 235],
        [236, 239, 238],
        [220, 226, 227]],

       ...,

       [[156, 167, 169],
        [165, 184, 192],
        [164, 183, 191],
        ...,
        [251, 253, 253],
        [251, 253, 253],
        [251, 253, 253]],

       [[165, 179, 183],
        [163, 182, 189],
        [163, 182, 189],
        ...,
        [251, 253, 253],
        [251, 253, 253],
        [251, 253, 253]],

       [[165, 179, 184],
        [164, 183, 191],
        [164, 183, 191],
        ...,
        [245, 248, 248],
        [245, 247, 247],
        [248, 250, 250]]

In [19]:
img_hor = np.hstack((img_output, stack_im, warped_copy))
cv2.imshow("Answers detection", img_hor)
cv2.waitKey(0)

-1

In [20]:
# Sort the question contours top-to-bottom, then initiaize the total number of correct answers
questionCnts = sorted(questionCnts, key=lambda x: cv2.boundingRect(x)[1])
correct = 0

In [21]:
# each question has 5 possible answers, to loop over the question in batches of 5
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

# Initialize the index of the bubbled answer
row_idx = 0

# Initialize an empty list to store the sorted contours
sorted_cnts = []

# Iterate over the contours in groups of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    # Sort the contours for the current question from left to right
    # If i is 0, the slice questionCnts[0:5] will contain the first 5 contours.
    cnts = sorted(questionCnts[i : i+5], key=lambda x: cv2.boundingRect(x)[0])
    sorted_cnts.extend(cnts)  # Add the sorted contours to the list

    # Update the row index
    row_idx += 1
    bubbled = None
    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current "bubble" for the question
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        # apply the mask to the thresholded image, then count the number of non-zero pixels in the bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        # if the current total has a larger number of total non-zero pixels, then we are examining the currently bubbled-in answer
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    # initialize the contour color and the index of the "correct" answer
    color = (0, 0, 255)
    k = ANSWER_KEY[q]
    # check to see if the bubbled answer is correct
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
    # draw the outline of the correct answer on the test
    cv2.drawContours(img_output, [cnts[k]], -1, color, 3)

# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(img_output, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", img_output)
cv2.waitKey(0)

[INFO] score: 80.00%


-1